## Resources on PLY

 
## Documentation of PLY is here: https://www.dabeaz.com/ply/ply.html
 

## This Youtube video is a talk by David Beazley, the author of PLY

In [ ]:
from IPython.display import YouTubeVideo
# Direct path is https://youtu.be/zJ9z6Ge-vXs
YouTubeVideo('zJ9z6Ge-vXs')

 ## This Youtube video is a talk on creating a calculator using PLY
 

In [ ]:
from IPython.display import YouTubeVideo
# Direct path is  https://youtu.be/Hh49BXmHxX8
YouTubeVideo('Hh49BXmHxX8')

# THIS NOTEBOOK is referenced in Asg-6
# It Creates a RegExp Parser + Draw Parse Trees

# YOUR QUESTIONS are at the end!

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import sys

# -- Detect if in Own Install or in Colab
try:
    import google.colab
    OWN_INSTALL = False
except:
    OWN_INSTALL = True
    
if OWN_INSTALL:
    
  #---- Leave these definitions ON if running on laptop
  #---- Else turn OFF by putting them between ''' ... '''

  sys.path[0:0] = ['../../../../..',  '../../../../../3rdparty',  
                   '../../../..',  '../../../../3rdparty',  
                   '../../..',     '../../../3rdparty', 
                   '../..',        '../../3rdparty',
                   '..',           '../3rdparty' ]

else: # In colab
  ! if [ ! -d Jove ]; then git clone https://github.com/ganeshutah/Jove Jove; fi
  sys.path.append('./Jove')
  sys.path.append('./Jove/jove')

# -- common imports --
from jove.lex import lex
from jove.yacc import yacc

from jove.StateNameSanitizers import ResetStNum, NxtStateStr
from jove.SystemImports       import *

from jove.Def_NFA import mk_nfa
from jove.DotBashers import *

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [ ]:
tokens = ('EPS','STR','LPAREN','RPAREN','PLUS','STAR')  

# Tokens in our RE are these
t_EPS     = r'\'\'|\"\"'  # Not allowing @ for empty string anymore! # t_EPS = r'\@'

# The following allows one lower-case, one upper-case or one digit to be used in our REs
t_STR     = r'[a-zA-Z0-9]'

t_LPAREN  = r'\('
t_RPAREN  = r'\)'

t_PLUS    = r'\+'
t_STAR    = r'\*'



# Ignored characters
t_ignore = " \t"

def t_newline(t):
    r'\n+'
    t.lexer.lineno += t.value.count("\n")
    
def t_error(t):
    print("Illegal character '%s'" % t.value[0])
    t.lexer.skip(1)
    

def p_expression_plus(t):
    'expression : expression PLUS catexp'
    #
    nfa = mk_plus_nfa(t[1]['nfa'], t[3]['nfa']) # Union of the two NFAs is returned
    tree = attrDyadicInfix("+", t[1], t[3])
    tree.update({'nfa':nfa})
    t[0] = tree
    

def p_expression_plus1(t):
    'expression : catexp'
    #
    t[0] = t[1]  

def p_expression_cat(t):
    'catexp :  catexp ordyexp'
    #
    nfa = mk_cat_nfa(t[1]['nfa'], t[2]['nfa'])
    #--insert new field 'nfa'
    tree = attrDyadicInfix(".", t[1], t[2])
    tree.update({'nfa':nfa})
    t[0] = tree

def p_expression_cat1(t):
    'catexp :  ordyexp'
    #
    t[0] = t[1] 

# We employ field 'ast' of the dict to record the abstract syntax tree. 
# Field 'dig' holds a digraph. It too is a dict. 
# Its fields are nl for the node list and el for the edge list

def p_expression_ordy_star(t):
    'ordyexp : ordyexp STAR'
    #
    nfa = mk_star_nfa(t[1]['nfa'])
    
    ast = ('*', t[1]['ast'])
    nlin = t[1]['dig']['nl']
    elin = t[1]['dig']['el']
    
    rootin = nlin[0]

    root = NxtStateStr("R*_") 
    right = NxtStateStr("*_")

    t[0] = {'nfa' : nfa,
            'ast' : ast,
            'dig' : {'nl' : [root] + nlin + [right], # this order important for proper layout!
                     'el' : elin + [ (root, rootin),
                                     (root, right) ]
                    }}
    
def p_expression_ordy_paren(t):
    'ordyexp : LPAREN expression RPAREN'
    #
    nfa  = t[2]['nfa']
    
    ast  = t[2]['ast']
    nlin = t[2]['dig']['nl']
    elin = t[2]['dig']['el']
    
    rootin = nlin[0]
    
    root = NxtStateStr("(R)_")
    left = NxtStateStr("(_")
    right= NxtStateStr(")_")
    
    t[0] = {'nfa' : nfa,
            'ast' : ast,
            'dig' : {'nl' : [root, left] + nlin + [right], #order important f. proper layout!
                     'el' : elin + [ (root, left),
                                     (root, rootin),
                                     (root, right) ]
                    }}
    
def p_expression_ordy_eps(t):
    'ordyexp : EPS'
    #
    strn = '@'
    ast  = ('@', strn)           
    t[0] = { 'nfa' : mk_eps_nfa(),
             'ast' : ast,
             'dig' : {'nl' : [ strn + NxtStateStr("_") ],
                      'el' : []
                     }} 
    
def p_expression_ordy_str(t):
    'ordyexp : STR'
    #
    str = t[1]
    nfa_STR = mk_symbol_nfa(t[1])
    ast  = ('str', str)
    t[0] = {'nfa' : nfa_STR,
            'ast' : ast,
            'dig' : {'nl' : [ str + NxtStateStr("_") ],
                     'el' : [] 
                    }}
    
def p_error(t):
    print("Syntax error at '%s'" % t.value)

#--
def parseRE(s):
    """In: a string s containing a regular expression.
       Out: An attribute quadruple (nfa,ast,nodelist,edgelist)        
    """
    mylexer  = lex()
    myparser = yacc()
    #-- pass the right lexer into the parser    
    p = myparser.parse(s, lexer = mylexer)  
    return (p['nfa'], p['ast'], p['dig']['nl'], p['dig']['el']) 

#--
def mk_plus_nfa(N1, N2):
    """Given two NFAs, return their union.
    """
    delta_accum = dict({})
    delta_accum.update(N1["Delta"])
    delta_accum.update(N2["Delta"]) # Simply accumulate the transitions
    # The alphabet is inferred bottom-up; thus we must union the Sigmas 
    # of the NFAs!
    return mk_nfa(Q     = N1["Q"] | N2["Q"], 
                  Sigma = N1["Sigma"] | N2["Sigma"], 
                  Delta = delta_accum, 
                  Q0    = N1["Q0"] | N2["Q0"], 
                  F     = N1["F"] | N2["F"])  

def mk_cat_nfa(N1, N2):
    '''Given two NFAs, return their concatenation.
    '''
    delta_accum = dict({}) 
    delta_accum.update(N1["Delta"])
    delta_accum.update(N2["Delta"])
    # Now, introduce moves from every one of N1's final states
    # to the set of N2's initial states.
    for f in N1["F"]:
        # However, N1's final states may already have epsilon moves to
        # other N1-states!
        # Expand the target of such jumps to include N2's Q0 also!
        if (f, "") in N1["Delta"]: 
            delta_accum.update({ (f,""):(N2["Q0"] | N1["Delta"][(f, "")])
                               })
        else:
            delta_accum.update({ (f, ""): N2["Q0"] })
    # In syntax-directed translation, it is impossible
    # that N2 and N1 have common states. Check anyhow
    # in case there are bugs elsewhere that cause it.
    assert((N2["F"] & N1["F"]) == set({})) 
    return mk_nfa(Q     = N1["Q"] | N2["Q"], 
                  Sigma = N1["Sigma"] | N2["Sigma"], 
                  Delta = delta_accum, 
                  Q0    = N1["Q0"],
                  F     = N2["F"])


def mk_star_nfa(N):
    '''Given an NFA, make its star.
    '''
    # Follow construction from Kozen's book:
    # 1) Introduce new (single) start+final state IF
    # 2) Let Q0 = set({ IF })
    # 2) Move on epsilon from IF to the set N[Q0]
    # 3) Make N[F] non-final
    # 4) Spin back from every state in N[F] to Q0
    #
    delta_accum = dict({})
    IF = NxtStateStr()
    Q0 = set({ IF }) # new set of start + final states
    # Jump from IF to N's start state set
    delta_accum.update({ (IF,""): N["Q0"] })
    delta_accum.update(N["Delta"])
    #
    for f in N["F"]:
        # N's final states may already have epsilon moves to
        # other N-states!
        # Expand the target of such jumps to include Q0 also.
        if (f, "") in N["Delta"]:
            delta_accum.update({ (f, ""): (Q0 | N["Delta"][(f, "")]) })
        else:
            delta_accum.update({ (f, ""): Q0 })
    #
    return mk_nfa(Q     = N["Q"] | Q0, 
                  Sigma = N["Sigma"], 
                  Delta = delta_accum, 
                  Q0    = Q0, 
                  F     = Q0)


def mk_eps_nfa():
    """An nfa with exactly one start+final state, which is the NFA for Epsilon.
    """
    Q0 = set({ NxtStateStr() })
    F  = Q0
    return mk_nfa(Q     = Q0, 
                  Sigma = set({}), 
                  Delta = dict({}), 
                  Q0    = Q0, 
                  F     = Q0)                      

def mk_symbol_nfa(a):
    """The NFA for a single re letter.
    """
    # Make a fresh initial state
    q0 = NxtStateStr()
    Q0 = set({ q0 })
    # Make a fresh final state
    f = NxtStateStr()
    F = set({ f })
    return mk_nfa(Q     = Q0 | F, 
                  Sigma = set({a}), 
                  Delta = { (q0,a): F },
                  Q0    = Q0, 
                  F     = F)
                                                        

In [ ]:
def attrDyadicInfix(op, attr1, attr3):         # <== this is what prints the parse-tree
    ast  = (op, (attr1['ast'], attr3['ast']))  # <== for an infix operator
    
    nlin1 = attr1['dig']['nl']
    nlin3 = attr3['dig']['nl']
    nlin  = nlin1 + nlin3
    
    elin1 = attr1['dig']['el']
    elin3 = attr3['dig']['el']
    elin  = elin1 + elin3
    
    rootin1 = nlin1[0]
    rootin3 = nlin3[0]    
    
    root   = NxtStateStr("R1"+op+"R2"+"_") # NxtStateStr("$_")
    left   = rootin1
    middle = NxtStateStr(op+"_")
    right  = rootin3
    
    return {'ast' : ast,
            'dig' : {'nl' : [ root, left, middle, right ] + nlin,
                     'el' : elin + [ (root, left),
                                     (root, middle),
                                     (root, right) ]
                     }}


def drawPT(nfa_ast_nl_el, comment="PT"):
    """Given an (nfa, ast, nl, el) quadruple where nl is the node and el the edge-list,
       draw the Parse Tree by returning a dot object. Also return the NFA dot object.
    """
    (nfa, ast, nl, el) = nfa_ast_nl_el
    print("Drawing AST for ", ast)
    dotObj_pt = Digraph(comment)
    dotObj_pt.graph_attr['rankdir'] = 'TB'
    for n in nl:
        prNam = n.split('_')[0]
        dotObj_pt.node(n, prNam, shape="oval", peripheries="1")
    for e in el:
        dotObj_pt.edge(e[0], e[1])
    return (dotObj_nfa(nfa), dotObj_pt)

# Now answer these questions

In [ ]:
parseRE("''")

In [ ]:
(n,t) = drawPT(parseRE("''"))

In [ ]:
n

In [ ]:
t

In [ ]:
(n1,t1) = drawPT(parseRE("(a*b*+cc)*"))

In [ ]:
(n2,t2) = drawPT(parseRE("(a*b)*+cc*"))

In [ ]:
n1

In [ ]:
t1

In [ ]:
n2

In [ ]:
t2


# YOUR QUESTIONS 

# Q1: Run this notebook as follows

## 1) Restart remove the parsetab.py file and __pycache__/

## 2) Run all the cells (ignore warnings such as this)

#### WARNING: ../../../../jove/TransitionSelectors.py:22: Possible grammar rule 'fn_range' defined without p_ prefix

## 3) Look at the productions (the ```p_``` functions) and write down a context-free grammar for parsing regular expressions

### YOUR ANSWER may please be written in this style, (inventing suitable abbreviations to express the High-level Rule)

* expression : expression PLUS catexpression
  - High-level Rule: R -> R + C  
  
* Do this for all the rules

* When you have things like LPAREN, look up how the token was encoded, and write `(` instead in the High-level Rule



# Q2: Execute these commands:

* (n1,t1) = drawPT(parseRE("(a*b*+cc)*"))

* (n2,t2) = drawPT(parseRE("(a*b)*+cc*"))

## By comparing n1 and t1, justify that the correct NFA formation rules have been followed

## Repeat for n2 and t2



# Q3: Explain the workings of all the mk_X_nfa functions by arguing that they are making the right output NFA from the input NFA. 

## In your explanation, explain how Q, Sigma, Delta, q0, and F are formed